In [1]:
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import spacy
from spacy import displacy

In [2]:
df = pd.read_pickle('../data/label_data.pk1')

In [333]:
spacy.prefer_gpu()

True

In [3]:
df['severity_h'] = df.label_h * df.score_h

In [16]:
#df = df[df.user != 'sneakpeek_bot']

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 174463 entries, 0 to 174462
Data columns (total 23 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   comment         174463 non-null  object        
 1   user            174463 non-null  object        
 2   date_time       174463 non-null  float64       
 3   sub_title       174463 non-null  object        
 4   clean_comments  174463 non-null  object        
 5   comment_len     174463 non-null  int64         
 6   entities        174463 non-null  object        
 7   polarity        174463 non-null  float64       
 8   subjectivity    174463 non-null  float64       
 9   title           174463 non-null  object        
 10  title_ents      174463 non-null  object        
 11  len_ents        174463 non-null  int64         
 12  entities_both   174463 non-null  object        
 13  len_entities    174463 non-null  int64         
 14  label_h         174463 non-null  int

In [46]:
df2 = df[df.entities_both.map(lambda x: len(set(x)))==1]

In [19]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149254 entries, 0 to 174462
Data columns (total 23 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   comment         149254 non-null  object        
 1   user            149254 non-null  object        
 2   date_time       149254 non-null  float64       
 3   sub_title       149254 non-null  object        
 4   clean_comments  149254 non-null  object        
 5   comment_len     149254 non-null  int64         
 6   entities        149254 non-null  object        
 7   polarity        149254 non-null  float64       
 8   subjectivity    149254 non-null  float64       
 9   title           149254 non-null  object        
 10  title_ents      149254 non-null  object        
 11  len_ents        149254 non-null  int64         
 12  entities_both   149254 non-null  object        
 13  len_entities    149254 non-null  int64         
 14  label_h         149254 non-null  int

In [20]:
unique_entities = pd.read_csv('../data/entity_list.csv')

In [21]:
unique_entities.columns = ['index', 'entity']

In [63]:
for item in tqdm(unique_entities.entity):
    rows = []
    for index,row in df2.iterrows():
        if item in row.entities_both:
            rows.append(row.values)
    item_df = pd.DataFrame(data = rows, columns = df.columns)
    item_df.entities = item
    item_df.to_pickle(f'../data/indiv_data/{item}.pk1')

100%|██████████| 114/114 [15:39<00:00,  8.24s/it]


In [22]:
df3 = df2[['clean_comments', 'user', 'label_h', 'entities']]

In [23]:
df3.to_pickle('../data/comment_truncated.pk1')

In [24]:
df3.head()

,clean_comments,user,label_h,entities
0,Seems like they misunderstood what is going on...,Worms_sg,0,[The Straits Times]
1,"On one hand, such research isn't necessarily a...",rcRollerCoaster,0,[Ministry of Education]
3,"The context is quite different leh. Norway, li...",Pyrrylanion,0,[Trace Together]
5,HK has more restrictions? Maybe lockdown style...,DuePomegranate,-1,[Trace Together]
6,> Commentary: Our fears are driving us to the...,sneakpeek_bot,0,[Ministry of Health]


In [25]:
X = df.clean_comments
df.label_v = [int(label) for label in df.label_v]
y_vader = df.label_v
y_hugging = df.label_h

In [26]:
X_train_h, X_test_h, y_train_h, y_test_h = train_test_split(X, y_hugging, test_size=200, random_state=42,
                                                            stratify=y_hugging)

X_train_v, X_test_v, y_train_v, y_test_v = train_test_split(X, y_vader, test_size=200, random_state=42,
                                                            stratify=y_vader)

In [27]:
df.head()

,comment,user,date_time,sub_title,clean_comments,comment_len,entities,polarity,subjectivity,title,...,len_entities,label_h,score_h,date,year_month,label_t,output_v,score_v,label_v,severity_h
0,Seems like they misunderstood what is going on...,Worms_sg,1.647150e+09,NTU used as example of university gaming ranki...,Seems like they misunderstood what is going on...,702,[The Straits Times],0.094773,0.293409,NTU used as example of university gaming ranki...,...,1,0,0.628446,2022-03-13 05:43:22,2022-03,1,"{'neg': 0.059, 'neu': 0.84, 'pos': 0.101, 'com...",0.5423,1,0.000000
1,"On one hand, such research isn't necessarily a...",rcRollerCoaster,1.647162e+09,NTU used as example of university gaming ranki...,"On one hand, such research isn't necessarily a...",2162,[Ministry of Education],0.063453,0.433367,NTU used as example of university gaming ranki...,...,1,0,0.480066,2022-03-13 08:57:22,2022-03,1,"{'neg': 0.055, 'neu': 0.793, 'pos': 0.152, 'co...",0.9789,1,0.000000
2,"PAP: And this is why, boys and girls, we saved...",nellynai,1.647143e+09,"Study: Iceland, Norway, and Singapore had no e...","PAP: And this is why, boys and girls, we saved...",288,"[Trace Together, People's Action Party]",0.283333,0.566667,"Study: Iceland, Norway, and Singapore had no e...",...,2,1,0.772632,2022-03-13 03:50:14,2022-03,1,"{'neg': 0.043, 'neu': 0.739, 'pos': 0.218, 'co...",0.9098,1,0.772632
3,"The context is quite different leh. Norway, li...",Pyrrylanion,1.647149e+09,"Study: Iceland, Norway, and Singapore had no e...","The context is quite different leh. Norway, li...",3196,[Trace Together],0.139835,0.524475,"Study: Iceland, Norway, and Singapore had no e...",...,1,0,0.521701,2022-03-13 05:20:02,2022-03,1,"{'neg': 0.063, 'neu': 0.764, 'pos': 0.173, 'co...",0.9968,1,0.000000
4,Also the early screening work done by the EDB....,Worms_sg,1.647159e+09,"Study: Iceland, Norway, and Singapore had no e...",Also the early screening work done by the EDB....,1525,"[Government, The Straits Times, Ministry of He...",0.083787,0.357625,"Study: Iceland, Norway, and Singapore had no e...",...,3,0,0.785580,2022-03-13 08:11:39,2022-03,1,"{'neg': 0.017, 'neu': 0.907, 'pos': 0.076, 'co...",0.9100,1,0.000000


In [ ]:
X_test_h = pd.read_csv('../data/hug_test.csv')
X_test_h.info()
y_true = X_test_h.label
print(classification_report(y_true, y_test_h, target_names=['negative', 'neutral', 'positive']))

In [ ]:
x_test_v = pd.read_csv('../data/vad_test.csv')
y_true_v = x_test_v.label

print(classification_report(y_true_v, y_test_v, target_names=['negative', 'neutral', 'positive']))

In [ ]:
ConfusionMatrixDisplay.from_predictions(y_true_v, y_test_v, cmap='Blues');
ConfusionMatrixDisplay.from_predictions(y_true, y_test_h, cmap='Blues');

## ClauCy Test

In [ ]:
claucy.add_to_pipe(nlp)
df_clean3 = df_clean2[df_clean2.entities_both.map(lambda x: len(x)) > 1]
df_clean3.info()

In [24]:
df_dp = pd.read_pickle('../data/indiv_data/Denise Phua.pk1')

In [25]:
df_dp

,comment,user,date_time,sub_title,clean_comments,comment_len,entities,polarity,subjectivity,title,...,len_entities,label_h,score_h,date,year_month,label_t,output_v,score_v,label_v,severity_h
0,i only remember Denise pua cb cos she open her...,nicky9499,1.646884e+09,Free public transport for vulnerable groups wi...,i only remember Denise pua cb cos she open her...,106,Denise Phua,0.000000,0.533333,Free public transport for vulnerable groups wi...,...,1,0,0.712950,2022-03-10 03:44:37,2022-03,0,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,0,0.000000
1,Denise Phua did help a bit to counter the nega...,fawe9374,1.646460e+09,"‘Never fudge or sugarcoat, never hide’: Joseph...",Denise Phua did help a bit to counter the nega...,55,Denise Phua,0.000000,0.000000,"‘Never fudge or sugarcoat, never hide’: Joseph...",...,1,0,0.695076,2022-03-05 05:53:26,2022-03,0,"{'neg': 0.0, 'neu': 0.769, 'pos': 0.231, 'comp...",0.4019,0,0.000000
2,Mayor Denise Phua?,blanketsforbitches,1.646484e+09,"‘Never fudge or sugarcoat, never hide’: Joseph...",Mayor Denise Phua?,18,Denise Phua,0.000000,0.000000,"‘Never fudge or sugarcoat, never hide’: Joseph...",...,1,0,0.865035,2022-03-05 12:36:30,2022-03,0,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,0,0.000000
3,Denise Phua got her aides to take down his con...,tryingmydarnest,1.644995e+09,Authorities blocked more than 350 websites in ...,Denise Phua got her aides to take down his con...,174,Denise Phua,-0.272778,0.404444,Authorities blocked more than 350 websites in ...,...,1,0,0.780115,2022-02-16 07:01:01,2022-02,-1,"{'neg': 0.064, 'neu': 0.936, 'pos': 0.0, 'comp...",-0.2960,-1,0.000000
4,"It irks me how confusing English actually is, ...",Environmental-Job363,1.642852e+09,Are there any regulations for these LED wreath...,"It irks me how confusing English actually is, ...",487,Denise Phua,0.002273,0.364394,Are there any regulations for these LED wreath...,...,1,-1,0.827834,2022-01-22 11:51:30,2022-01,0,"{'neg': 0.016, 'neu': 0.884, 'pos': 0.1, 'comp...",0.8338,0,-0.827834
5,">Unable to get through, Denise’s husband carri...",junkredpuppy,1.642171e+09,"OCBC S'pore scam victims, many who lost life s...",">Unable to get through, Denise’s husband carri...",283,Denise Phua,-0.016667,0.316667,"OCBC S'pore scam victims, many who lost life s...",...,1,-1,0.829508,2022-01-14 14:38:47,2022-01,0,"{'neg': 0.049, 'neu': 0.904, 'pos': 0.046, 'co...",-0.0380,0,-0.829508
6,Denise will represent all the other mayors and...,jmzyn,1.640958e+09,Singapore to ‘start moving’ on planned GST hik...,Denise will represent all the other mayors and...,103,Denise Phua,-0.125000,0.375000,Singapore to ‘start moving’ on planned GST hik...,...,1,0,0.763015,2021-12-31 13:46:37,2021-12,-1,"{'neg': 0.129, 'neu': 0.746, 'pos': 0.124, 'co...",-0.0258,-1,0.000000
7,Actually what happened to that ah? Besides Den...,CisternOfADown,1.641014e+09,Singapore to ‘start moving’ on planned GST hik...,Actually what happened to that ah? Besides Den...,110,Denise Phua,0.200000,0.500000,Singapore to ‘start moving’ on planned GST hik...,...,1,0,0.851917,2022-01-01 05:05:16,2022-01,1,"{'neg': 0.0, 'neu': 0.896, 'pos': 0.104, 'comp...",0.2168,1,0.000000
8,"Only one of the 4 or 5 is full time, Denise Ph...",tongzhimen,1.640744e+09,Government Not Awarding Prize Money For Loh Ke...,"Only one of the 4 or 5 is full time, Denise Ph...",91,Denise Phua,0.175000,0.775000,Government Not Awarding Prize Money For Loh Ke...,...,1,-1,0.516411,2021-12-29 02:21:03,2021-12,1,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,1,-0.516411
9,"Oh don’t worry, it very much lives on as the b...",fitzerspaniel,1.640694e+09,Government Not Awarding Prize Money For Loh Ke...,"Oh don’t worry, it very much lives on as the b...",173,Denise Phua,0.130000,0.630000,Government Not Awarding Prize Money For Loh Ke...,...,1,0,0.522727,2021-12-28 12:12:13,2021-12,1,"{'neg': 0.123, 'neu': 0.744, 'pos': 0.133, 'co...",0.1027,1,0.000000


In [6]:

nlp = spacy.load('../base_model/model-last')

In [7]:
nlp.pipe_names

['transformer', 'tagger', 'parser', 'ner']

In [6]:
#nlp.to_disk('../model')

In [7]:
df_at = pd.read_pickle('../data/indiv_data/Alvin Tan.pk1')

In [8]:
df_at

,comment,user,date_time,sub_title,clean_comments,comment_len,entities,polarity,subjectivity,title,...,len_entities,label_h,score_h,date,year_month,label_t,output_v,score_v,label_v,severity_h
0,How did Min. Alvin Tan get into the University...,sunburnt258,1.645585e+09,/r/singapore random discussion and small quest...,How did Min. Alvin Tan get into the University...,93,Alvin Tan,0.000000,0.000000,/r/singapore random discussion and small quest...,...,1,0,0.897489,2022-02-23 02:52:46,2022-02,0,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,0,0.000000
1,"Yeah, same as Alvin Tan. I have no idea how th...",nellynai,1.634608e+09,"S'pore's Covid-19 strategy not a 'flip-flop', ...","Yeah, same as Alvin Tan. I have no idea how th...",69,Alvin Tan,0.000000,0.125000,"S'pore's Covid-19 strategy not a 'flip-flop', ...",...,1,-1,0.708544,2021-10-19 01:40:51,2021-10,0,"{'neg': 0.126, 'neu': 0.747, 'pos': 0.126, 'co...",0.0000,0,-0.708544
2,Alvin Tan?,Dr-Vijay,1.625848e+09,Singtel joins growing list of firms in disclos...,Alvin Tan?,10,Alvin Tan,0.000000,0.000000,Singtel joins growing list of firms in disclos...,...,1,0,0.850523,2021-07-09 16:20:09,2021-07,0,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,0,0.000000
3,"Summary: Yes, it’s OK to feel nervous with eve...",mtb22,1.625620e+09,Alvin Tan on Imposter Syndrome,"Summary: Yes, it’s OK to feel nervous with eve...",147,Alvin Tan,0.347273,0.550909,Alvin Tan on Imposter Syndrome,...,1,1,0.515519,2021-07-07 01:01:43,2021-07,1,"{'neg': 0.065, 'neu': 0.678, 'pos': 0.257, 'co...",0.7378,1,0.515519
4,Doesn’t it feel like a bit of a humblebrag? Ma...,law90026,1.625626e+09,Alvin Tan on Imposter Syndrome,Doesn’t it feel like a bit of a humblebrag? Ma...,67,Alvin Tan,0.800000,0.700000,Alvin Tan on Imposter Syndrome,...,1,0,0.688839,2021-07-07 02:54:49,2021-07,1,"{'neg': 0.0, 'neu': 0.694, 'pos': 0.306, 'comp...",0.6486,1,0.000000
5,For anyone who doesn't feel like clicking thro...,Varantain,1.625680e+09,Alvin Tan on Imposter Syndrome,For anyone who doesn't feel like clicking thro...,2440,Alvin Tan,0.224739,0.467424,Alvin Tan on Imposter Syndrome,...,1,-1,0.475560,2021-07-07 17:50:36,2021-07,1,"{'neg': 0.07, 'neu': 0.813, 'pos': 0.117, 'com...",0.9845,1,-0.475560
6,What do you call the syndrome when you get in ...,thrulim123,1.625631e+09,Alvin Tan on Imposter Syndrome,What do you call the syndrome when you get in ...,137,Alvin Tan,0.066667,0.566667,Alvin Tan on Imposter Syndrome,...,1,-1,0.802090,2021-07-07 04:13:04,2021-07,1,"{'neg': 0.113, 'neu': 0.784, 'pos': 0.103, 'co...",-0.0772,1,-0.802090
7,Being a sussy impostor 😳😳😳😳😳 among us referenc...,GreySkyRain,1.625625e+09,Alvin Tan on Imposter Syndrome,Being a sussy impostor 😳😳😳😳😳 among us referenc...,99,Alvin Tan,-0.400000,0.400000,Alvin Tan on Imposter Syndrome,...,1,-1,0.632056,2021-07-07 02:24:11,2021-07,-1,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,-1,-0.632056
8,"Ooooh, I thought it was that Alvin Tan.",mayorduke,1.625749e+09,Alvin Tan on Imposter Syndrome,"Ooooh, I thought it was that Alvin Tan.",39,Alvin Tan,0.000000,0.000000,Alvin Tan on Imposter Syndrome,...,1,0,0.715111,2021-07-08 12:48:56,2021-07,0,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,0,0.000000
9,LinkedIn ma. Is it really a LinkedIn post if y...,thinkingpanda,1.625633e+09,Alvin Tan on Imposter Syndrome,LinkedIn ma. Is it really a LinkedIn post if y...,68,Alvin Tan,0.000000,0.300000,Alvin Tan on Imposter Syndrome,...,1,0,0.615418,2021-07-07 04:51:02,2021-07,0,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,0,0.000000


In [5]:
df2 = df[df.len_entities>1]

In [437]:
df2.loc[80064]

comment           Kudos to Mothership. OYK should fund them with...
user                                                        mrwagga
date_time                                              1632932001.0
sub_title         Mothership infographic on what to do if you ar...
clean_comments    Kudos to Mothership. OYK should fund them with...
comment_len                                                      67
entities                                  [Ong Ye Kung, Mothership]
polarity                                                        0.0
subjectivity                                                    0.0
title             Mothership infographic on what to do if you ar...
title_ents                                             [Mothership]
len_ents                                                          2
entities_both                             [Ong Ye Kung, Mothership]
len_entities                                                      2
label_h                                         

In [7]:
df3 = df[df.entities.map(lambda x:len(x))>1]

In [8]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9486 entries, 2 to 174446
Data columns (total 23 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   comment         9486 non-null   object        
 1   user            9486 non-null   object        
 2   date_time       9486 non-null   float64       
 3   sub_title       9486 non-null   object        
 4   clean_comments  9486 non-null   object        
 5   comment_len     9486 non-null   int64         
 6   entities        9486 non-null   object        
 7   polarity        9486 non-null   float64       
 8   subjectivity    9486 non-null   float64       
 9   title           9486 non-null   object        
 10  title_ents      9486 non-null   object        
 11  len_ents        9486 non-null   int64         
 12  entities_both   9486 non-null   object        
 13  len_entities    9486 non-null   int64         
 14  label_h         9486 non-null   int64         
 15  sc

In [9]:
nlp2 = spacy.load('../base_model/model-last')

In [10]:
comments = [comment for comment in df3.clean_comments[560:1070]]

In [11]:
comments

['So many salty comments about VB and TT. Is this the best excuse there is? Sheesh.',
 'Please setup a committee to check Vivian lied about Trace Together. Why he admit about it and that about it. No further action. Then why RK admit lying and was send to committee.',
 'I think most PAP leaders are like Singaporeans. We just want things to work. As for MPs who adopt LHL’s line of thinking… I really hope they retire sooner than later',
 'Come come WP simps let me continue breaking down your little fantasy bubble for your since so many ppl are saying "Oh its just a small issue lets get it over";"ohh VB lied about TT"; "Oh PAP is making this political there is no evidence u cant trust a liar RS" First off, lying in parliament is not a small issue, especially when the allegations as about PS instructing RS to keep lying. Assuming he didn\'t, all that is going to happen is a few months more for PS to prove himself before this entire saga ends. Assuming he DID instruct RS to lie, do you know

In [12]:
nlp3 = spacy.load('en_core_web_trf')

In [13]:
nlp.pipe_names

NameError: name 'nlp' is not defined

In [14]:
nlp2.pipe_names

['transformer', 'tagger', 'parser', 'ner']

In [15]:
nlp3.pipe_names

['transformer', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [35]:
options = {'compact': True, 'fine_grained':  True, 'collapse_punct': True}
for comment in comments[0:2]:
    doc = nlp2(comment)
    sentence_spans = list(doc.sents)
    displacy.render(doc, style = 'dep', options = options)

In [145]:
nlp4 = spacy.load('../model')

In [411]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 174463 entries, 0 to 174462
Data columns (total 23 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   comment         174463 non-null  object        
 1   user            174463 non-null  object        
 2   date_time       174463 non-null  float64       
 3   sub_title       174463 non-null  object        
 4   clean_comments  174463 non-null  object        
 5   comment_len     174463 non-null  int64         
 6   entities        174463 non-null  object        
 7   polarity        174463 non-null  float64       
 8   subjectivity    174463 non-null  float64       
 9   title           174463 non-null  object        
 10  title_ents      174463 non-null  object        
 11  len_ents        174463 non-null  int64         
 12  entities_both   174463 non-null  object        
 13  len_entities    174463 non-null  int64         
 14  label_h         174463 non-null  int

In [432]:
doc = nlp4(df.clean_comments[80064])

In [433]:
spacy.explain('nsubjpass')

'nominal subject (passive)'

In [434]:
displacy.render(doc, style = 'dep', options = options)

In [435]:
print(doc.text)

for ents in doc.ents:
    for token in ents:
        if token.ent_iob == 3:
            clause = []
            current_token = token
            subject_count = 0
            while current_token.dep_ != "ROOT" and subject_count <= 1:
                if current_token.dep_ ==  'nsubj' or current_token.dep_== 'iobj' or current_token.dep_== 'dobj' or current_token.dep_ == 'pobj' or current_token.dep_ == 'nsubjpass':
                    subject_count += 1
                    current_token = current_token.head
                else:
                    current_token = current_token.head
            print([t for t in current_token.subtree]+[ents.kb_id_])

Kudos to Mothership. OYK should fund them with some of his pay tbh.
[Kudos, to, Mothership, ., 'Mothership']
[OYK, should, fund, them, with, some, of, his, pay, tbh, ., 'Ong Ye Kung']


In [373]:
comment_dict = {}
count = 0
for doc in tqdm(nlp4.pipe(comments[0:50])):
    comment_dict[f'{count}'] = {'comment': doc.text}
    relevant_sections = {}
    for ents in doc.ents:
        relevant_sections[f'{ents.kb_id_}'] = []
        for token in ents:
            if token.ent_iob == 3:
                clause = {}
                current_token = token
                subject_count = 1
                while current_token.dep_ != "ROOT" and subject_count <= 1:
                    if current_token.dep_ ==  'nsubj' or current_token.dep_== 'iobj' or current_token.dep_== 'dobj' or current_token.dep_ == 'pobj':
                        subject_count += 1
                        current_token = current_token.head
                    else:
                        current_token = current_token.head
        section = [t for t in current_token.subtree]
        relevant_sections[f'{ents.kb_id_}'].append(section)
        comment_dict[f'{count}']['entities'] = relevant_sections
    count+= 1


50it [00:43,  1.14it/s]


In [438]:
comment_dict

{'0': {'comment': 'So many salty comments about VB and TT. Is this the best excuse there is? Sheesh.',
  'entities': {'Vivian Balakrishnan': [[about, VB, and, TT]],
   'Trace Together': [[about, VB, and, TT]]}},
 '1': {'comment': 'Please setup a committee to check Vivian lied about Trace Together. Why he admit about it and that about it. No further action. Then why RK admit lying and was send to committee.',
  'entities': {'Vivian Balakrishnan': [[Vivian, lied, about, Trace, Together]],
   'Trace Together': [[about, Trace, Together]],
   'Raeesah Khan': [[Then,
     RK,
     admit,
     lying,
     and,
     why,
     was,
     send,
     to,
     committee,
     .]]}},
 '2': {'comment': 'I think most PAP leaders are like Singaporeans. We just want things to work. As for MPs who adopt LHL’s line of thinking… I really hope they retire sooner than later',
  'entities': {"People's Action Party": [[most,
     PAP,
     leaders,
     are,
     like,
     Singaporeans]],
   'Lee Hsien Loong'

In [ ]:
comments[0:10]

In [ ]:
            token_ancestors = [t for t in token.ancestors]
            clause = token_ancestors
            for t in token.subtree:
                clause = clause + [t]
            clause = clause + [{ents:ents.kb_id_}]
            print(clause)

In [ ]:
    if token.ent_iob == 3:
        token_ancestors = [t.text for t in token.ancestors]
        print(token_ancestors)
        print(token.ent_id_)
        clause = [token]
        current_token = token
        while current_token.dep_ != "ROOT":
            clause = [current_token.head]+ clause
            current_token = current_token.head
        #for token in token.subtree:
            #clause.append(token)
        #print(clause)



        #root = token.head

        #print(f'{token.dep_}, {token.head.text}, {token.text}, {[child for child in token.children]} ')
    #if token.dep_ == 'nsubj' or token.dep_ == 'pobj' or token.dep_ =='dobj':
        #print(token.text)

In [246]:
for ents in doc.ents:
    for token in ents:
        if token.ent_iob == 3:
            span = doc[doc[token.i].left_edge.i : doc[token.i].right_edge.i+1]


Shan
RK's
WP


In [115]:
for token in doc:
    print(token.dep_)

punct
det
npadvmod
punct
det
compound
nsubj
ROOT
advmod
det
attr
prep
amod
pobj
cc
conj
prep
poss
amod
pobj
nsubj
aux
advmod
relcl
aux
xcomp
punct
nsubj
aux
advmod
ROOT
aux
xcomp
punct
det
amod
nsubj
punct
ROOT
prep
pobj
acl
intj
dobj
prep
poss
case
punct
nmod
nmod
punct
pobj
punct
compound
compound
compound
conj
cc
compound
compound
conj
advmod
advmod
conj
cc
advmod
conj
prep
compound
compound
compound
pobj
punct
appos
cc
compound
compound
conj
prep
advmod
advmod
nsubj
pcomp
prep
intj
pobj
cc
det
conj
punct
nsubj
advmod
prep
det
pobj
cc
det
amod
conj
prep
pobj
ROOT
mark
punct
advmod
punct
nsubjpass
auxpass
neg
ccomp
prt
agent
compound
pobj
cc
conj
punct
nmod
cc
conj
nsubj
aux
ROOT
det
dobj
punct
aux
advcl
prep
pobj
npadvmod
punct
nsubj
ROOT
aux
dep
xcomp
punct
dep
prt
det
amod
dobj
cc
conj
det
dobj
mark
nsubj
aux
advcl
dobj
advmod
punct
nsubj
aux
neg
advmod
ROOT
det
dobj
prep
pcomp
prep
det
pobj
punct
nummod
npadvmod
advmod
compound
poss
case
nsubjpass
prep
det
pobj
aux
aux
auxpass
RO

'direct object'

In [134]:
options = {'compact': True, 'fine_grained': True, 'collapse_punct': True}


sentence_spans = list(doc.sents)
displacy.render(doc, style = 'ent', options = options)

In [155]:
for comment in comments[0:2]:
    doc = nlp(comment)
    subjects = [token for token in doc if token.ent_iob == 3]
    for subject in subjects:
        print(subject)
        print(subject.dep_)



VB
npadvmod
TT
ROOT
Vivian
ROOT
Trace
ROOT


In [162]:
nlp.remove_pipe('transformer')

('transformer',
 <spacy_transformers.pipeline_component.Transformer at 0x2043bedb6a0>)

In [163]:
nlp.pipe_names

['tagger', 'parser', 'attribute_ruler', 'ner', 'entity_linker']

In [164]:
nlp.add_pipe('transformer', source = nlp2, before = 'tagger')

In [165]:
options = {'compact': True, 'fine_grained': False, 'collapse_punct': True}
for comment in comments[0:2]:
    doc = nlp(comment)
    sentence_spans = list(doc.sents)
    displacy.render(doc, style = 'dep', options = options)

In [166]:
options = {'compact': True, 'fine_grained': False, 'collapse_punct': True}
for comment in comments[0:2]:
    doc = nlp(comment)
    sentence_spans = list(doc.sents)
    displacy.render(doc, style = 'ent', options = options)

E:\anaconda3\envs\spacyenv\lib\site-packages\spacy\displacy\__init__.py:200: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)
